In [15]:
import openai
from dotenv import load_dotenv
import os
import json
import time

### Objetivo: Dada una clusterizacion de habilidades, el LLM etiqueta cada cluster con base en sus elementos
Se usa Llama para este ejercicio

**Consideraciones:**

- Usar una api gratuita excede el límite de tokens que se pueden enviar por minuto
- Los clusters actuales tienen una cardinalidad grande por lo cual limita el procesamiento mediante API gratuita. Por lo tanto para este ejercicio estoy tomando una muestra.


In [18]:
# Cargar variables de entorno
load_dotenv()

True

In [12]:
#cargar datos
with open("../output/cluster_skills.json", "r") as f:
    cluster_skills = {int(k): v for k, v in json.load(f).items()}

### LLM

In [13]:
# Leer la API Key
api_key = os.getenv("GROQ_API_KEY")

### Ejemplo de cómo llamar al LLM

In [4]:
openai.api_base = "https://api.groq.com/openai/v1"
openai.api_key = api_key


response = openai.ChatCompletion.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": "Dame un nombre de categoría para: Python, Java, C++."}]
)

print(response["choices"][0]["message"]["content"])


Una categoría común para esos lenguajes de programación es:

**Lenguajes de Programación Orientados a Objetos**

(Podría también ser simplemente "Lenguajes de Programación" o "Lenguajes de Alto Nivel", pero la categoría de "Orientados a Objetos" resalta una característica común entre Python, Java y C++).


### Utilizar el LLM para nombrar los clusters

In [17]:
# Configurar la API de Groq
openai.api_base = "https://api.groq.com/openai/v1"
openai.api_key = api_key  # asegúrate de que esté definida

# Diccionario donde guardaremos los resultados
cluster_categories = {}

# Límite de habilidades por cluster (opcional)
max_skills = 100

# Iterar sobre cada cluster
for cluster_id, skills in cluster_skills.items():
    if not skills:
        continue

    # Usar solo las primeras N (max_skills) habilidades para evitar mensajes largos
    skill_string = ", ".join(skills[:max_skills])
    prompt = f"Dame un nombre de categoría de habilidades laborales para el siguiente listado: {skill_string}."

    try:
        response = openai.ChatCompletion.create(
            model="llama3-70b-8192",
            messages=[{"role": "user", "content": prompt}]
        )

        category_name = response["choices"][0]["message"]["content"].strip()
        cluster_categories[cluster_id] = category_name
        print(f"Cluster {cluster_id}: {category_name}\n")

    except Exception as e:
        print(f"Error con el cluster {cluster_id}: {e}")

    # Esperar 1 segundo para evitar sobrepasar los límites
    time.sleep(1)

# Guardar resultados
with open("cluster_categories.json", "w") as f:
    json.dump(cluster_categories, f, indent=2, ensure_ascii=False)


Cluster 7: Después de revisar el listado, puedo sugerir algunas categorías de habilidades laborales que agrupen estas habilidades. A continuación, te presento algunas opciones:

1. **Mantenimiento y Reparación**: mantenimiento de maquinaria, manejo de bombas de vacío, mecánica hidráulica, mantenimiento preventivo, reparación de máquinas y equipos, etc.
2. **Habilidades Técnicas**: manejo de equipos de soldadura, manejo de equipos de proceso, control de aguas, control de emisiones atmosféricas, sellado de fluidos, etc.
3. **Gestión y Administración**: gestora y consultora asesora, planeación de compras y contratación, cobro de cartera, licencias A2 y C1, etc.
4. **Seguridad y Medio Ambiente**: desinsectación, lavado y desinfección de tanques, manejo de residuos, emergencias, seguridad vial, etc.
5. **Habilidades Informáticas**: manejo de herramientas ofimáticas, Herramienta SAP, reporte de daños en máquinas y equipos, etc.
6. **Construcción y Mantenimiento de Infraestructura**: construc